In [187]:
#Función para cargar archivos y devolver listas de transacciones
def cargar_datos(archivo):
    with open(archivo) as f:
        contenido = f.readlines()

    contenido = [x.strip() for x in contenido]
    Transaccion = []

    for i in range(0, len(contenido)):
        Transaccion.append(contenido[i].split())

    return Transaccion

#Para convertir la transacción inicial y mantenerla frozenset
def Set_inicio(dataset):
    retDict = {}
    for trans in dataset:
        retDict[frozenset(trans)] = 1
    return retDict

#clase de nodo arbol FP
class nodos_arbol:
    def __init__(self, Nodo,conteo,nodoPadre):
        self.nodoC = Nodo
        self.count = conteo
        self.nodoLiga = None
        self.padre = nodoPadre
        self.hijo = {}

    def incremento_contador(self, contador):
        self.count += contador

#Para crear Raiz_arbol y itemsets ordenados para arbol FP
def crear_FP_Arbol(dataset, minSoport):
    Raiz_arbol = {}
    for transaccion in dataset:
        for item in transaccion:
            Raiz_arbol[item] = Raiz_arbol.get(item,0) + dataset[transaccion]
    for k in list(Raiz_arbol):
        if Raiz_arbol[k] < minSoport:
            del(Raiz_arbol[k])

    frecuent_itemset = set(Raiz_arbol.keys())

    if len(frecuent_itemset) == 0:
        return None, None

    for k in Raiz_arbol:
        Raiz_arbol[k] = [Raiz_arbol[k], None]

    retTree = nodos_arbol('Null Set',1,None)
    for itemset,contar in dataset.items():
        frecuen_transaccion = {}
        for item in itemset:
            if item in frecuent_itemset:
                frecuen_transaccion[item] = Raiz_arbol[item][0]
        if len(frecuen_transaccion) > 0:
            ordenar_itemset = [v[0] for v in sorted(frecuen_transaccion.items(), key=lambda p: p[1], reverse=True)]
            #actualizar FPTree
            actualizarArbol(ordenar_itemset, retTree, Raiz_arbol, contar)
    return retTree, Raiz_arbol

def actualizarArbol(itemset, FPTree, Raiz_arbol, count):
    if itemset[0] in FPTree.hijo:
        FPTree.hijo[itemset[0]].incremento_contador(count)
    else:
        FPTree.hijo[itemset[0]] = nodos_arbol(itemset[0], count, FPTree)

        if Raiz_arbol[itemset[0]][1] == None:
            Raiz_arbol[itemset[0]][1] = FPTree.hijo[itemset[0]]
        else:
            actualizar_nodo(Raiz_arbol[itemset[0]][1], FPTree.hijo[itemset[0]])

    if len(itemset) > 1:
        actualizarArbol(itemset[1::], FPTree.hijo[itemset[0]], Raiz_arbol, count)


def actualizar_nodo(Nodo_test, nodo_objetivo):
    while (Nodo_test.nodoLiga != None):
        Nodo_test = Nodo_test.nodoLiga

    Nodo_test.nodoLiga = nodo_objetivo


def FPTree_transversal(nodo_hoja, ruta_opcion):
 if nodo_hoja.padre != None:
    ruta_opcion.append(nodo_hoja.nodoC)
    FPTree_transversal(nodo_hoja.padre, ruta_opcion)

def encontrar_ruta(basePat, nodos_arbol):
 patron_cond_base = {}

 while nodos_arbol != None:
    ruta_opcion = []
    FPTree_transversal(nodos_arbol, ruta_opcion)
    if len(ruta_opcion) > 1:
        patron_cond_base[frozenset(ruta_opcion[1:])] = nodos_arbol.count
    nodos_arbol = nodos_arbol.nodoLiga

 return patron_cond_base


def Buscar_Tree(FPTree, indicarTabla, minSuport, prefix, frecuent_itemset):
    bigL = [v[0] for v in sorted(indicarTabla.items(),key=lambda p: p[1][0])]
    for basePat in bigL:
        new_frecuentset = prefix.copy()
        new_frecuentset.add(basePat)

        frecuent_itemset.append(new_frecuentset)

        Patron_base_condi = encontrar_ruta(basePat, indicarTabla[basePat][1])

        Condicional_FPTree, Cond_cabecera = crear_FP_Arbol(Patron_base_condi,minSuport)

        if Cond_cabecera != None:
            Buscar_Tree(Condicional_FPTree, Cond_cabecera, minSuport, new_frecuentset, frecuent_itemset)

archivo = 'datos2.txt'
print("Mínimo conteo de soporte:")
soporte_minimo = 3
print(soporte_minimo)
initSet = Set_inicio(cargar_datos(archivo))
FPtree, indicarTabla = crear_FP_Arbol(initSet, soporte_minimo)

frecuent_itemset = []

Buscar_Tree(FPtree, indicarTabla, soporte_minimo, set([]), frecuent_itemset)

print("Todas las frecuencias de los itemsets:")
print(frecuent_itemset)

Mínimo conteo de soporte:
3
Todas las frecuencias de los itemsets:
[{'A'}, {'B', 'A'}, {'C'}, {'D', 'C'}, {'B', 'C'}, {'D'}, {'D', 'B'}, {'B'}]


In [181]:
from itertools import combinations, chain
from collections import defaultdict

def obtener_transacciones(datos):
    transacciones = []
    for dato in datos:
        transacciones.append(set(dato))
    return transacciones

def obtener_itemsets_frecuentes(transacciones, itemsets, min_support):
    item_counts = defaultdict(int)
    for transaccion in transacciones:
        for item in itemsets:
            if item.issubset(transaccion):
                item_counts[item] += 1

    num_transacciones = len(transacciones)
    itemsets_frecuentes = []
    for item, count in item_counts.items():
        support = count / num_transacciones
        if support >= min_support:
            itemsets_frecuentes.append((item, support))
    
    return itemsets_frecuentes

def generar_reglas_asociacion(itemsets_frecuentes, min_confidence):
    reglas_asociacion = []
    for i in range(1, len(itemsets_frecuentes)):
        for itemset, support in itemsets_frecuentes[i]:
            for j in range(1, i):
                for antecedente in combinations(itemset, j):
                    consecuente = itemset.difference(antecedente)
                    confidence = support / next(supp for item, supp in itemsets_frecuentes if item == set(antecedente))
                    if confidence >= min_confidence:
                        reglas_asociacion.append(((antecedente, consecuente), confidence))
    return reglas_asociacion

# Datos
datos = [
    ['A', 'B', 'C', 'D'],
    ['A', 'B', 'D'],
    ['A', 'B'],
    ['B', 'C', 'D'],
    ['B', 'C'],
    ['C', 'D'],
    ['B', 'D']
]

# Parámetros
min_support = 0.3
min_confidence = 0.5

# Obtener transacciones
transacciones = obtener_transacciones(datos)

# Generar itemsets frecuentes
items = set(chain(*transacciones))
itemsets_frecuentes = []
for i in range(1, len(items)+1):
    itemsets = set(combinations(items, i))
    itemsets_frecuentes.extend(obtener_itemsets_frecuentes(transacciones, itemsets, min_support))

# Generar reglas de asociación
reglas = generar_reglas_asociacion(itemsets_frecuentes, min_confidence)

# Mostrar resultados
print("Itemsets frecuentes:")
for itemset, support in itemsets_frecuentes:
    print(f"{itemset} - Soporte: {support}")

print("\nReglas de asociación:")
for regla, confidence in reglas:
    ante, conse = regla
    print(f"{ante} => {conse} - Confianza: {confidence}")



AttributeError: 'tuple' object has no attribute 'issubset'

In [ ]:
import numpy as np
 
def Apriori_gen(Itemset, tamano):
    """Generar nuevos (k+1)-itemsets"""
    candidatos = []
    for i in range(0, tamano):
        element = Itemset[i]
        for j in range(i + 1, tamano):
            element1 = Itemset[j]
            if element[:-1] == element1[:-1]:
                unionset = element + element1[-1]
                candidatos.append(unionset)
    return candidatos
 
def Apriori_prune(Ck,MinSoport):
    L = []
    for i in Ck:
        if Ck[i] >= minsoport:
            L.append(i)
    return sorted(L)
 
def Apriori_conteo_subset(Candidato,Candidato_tamano):
    Lk = dict()
    archivo = open('datos3.txt')
    for l in archivo:
        l = str(l.split())
        conteo = 0
        for i in range (0,Candidato_tamano):
            habilitar = str(Candidato[i])
            if habilitar not in Lk:
                Lk[habilitar] = 0
            bandera = True
            for k in habilitar:
                if k not in l:
                    bandera = False
            if bandera:
                Lk[habilitar] += 1
    archivo.close()
    return Lk
 
minsoport = 2
C1={}
archivo = open('datos3.txt')
 
for linea in archivo:
    for item in linea.split():
        if item in C1:
            C1[item] +=1
        else:
            C1[item] = 1
archivo.close()
print(C1)
L = []
L1 = Apriori_prune(C1,minsoport)
L = Apriori_gen(L1,len(L1))
print ('*************************************')
print ('Frecuencia 1-itemset ',L1)
print ('*************************************')
k=2
while L != []:
    C = dict()
    C = Apriori_conteo_subset(L,len(L))
    frecuencia_itemset = []
    frecuencia_itemset = Apriori_prune(C,minsoport)
    print ('----------------------------------')
    print ('Frecuencia',k,'-itemset ',frecuencia_itemset)
    print ('----------------------------------')
    L = Apriori_gen(frecuencia_itemset,len(frecuencia_itemset))
    k += 1

{'I1': 4, 'I2': 5, 'I3': 4, 'I4': 4, 'I5': 2}
*************************************
Frecuencia 1-itemset  ['I1', 'I2', 'I3', 'I4', 'I5']
*************************************
----------------------------------
Frecuencia 2 -itemset  ['I12', 'I13', 'I14', 'I23', 'I24', 'I34']
----------------------------------
----------------------------------
Frecuencia 3 -itemset  ['I123', 'I124', 'I234']
----------------------------------
----------------------------------
Frecuencia 4 -itemset  []
----------------------------------


In [ ]:
from mlxtend.frequent_patterns import association_rules

# Definir el diccionario de frecuencia del conjunto de elementos
itemset = {'I1': 4, 'I2': 5, 'I3': 4, 'I4': 4, 'I5': 2}

# Convertir el diccionario en un DataFrame
df = pd.DataFrame(list(itemset.items()), columns=['item', 'support'])

# Calcular las reglas de asociación con su confianza
rules = association_rules(df, metric="confidence", min_threshold=0.5)

# Mostrar las reglas de asociación y su confianza
print(rules)

ValueError: Dataframe needs to contain the                         columns 'support' and 'itemsets'

In [ ]:
salida = {'A': 3, 'B': 6, 'C': 4, 'D': 5}
def crear_arbol_salida(salida):
    retTree = nodos_arbol('Null Set', 1, None)  # Crea un árbol FP inicial
    Raiz_arbol = {item: [count, None] for item, count in salida.items()}  # Crea el diccionario Raiz_arbol
    for item, count in salida.items():  # Agrega cada elemento al árbol con su frecuencia
        actualizarArbol([item], retTree, Raiz_arbol, count)
    return retTree  # Devuelve el árbol creado

# Llama a la función para crear el árbol a partir de la salida
arbol_salida = crear_arbol_salida(salida)

# Función para imprimir el árbol
def imprimir_arbol(arbol, nivel=0):
    if arbol.nodoC != 'Null Set':
        print('  ' * nivel, arbol.nodoC, ' ', arbol.count)
    if arbol.hijo:
        for nodo_hijo in arbol.hijo.values():
            imprimir_arbol(nodo_hijo, nivel + 1)

# Imprimir el árbol resultante
imprimir_arbol(arbol_salida)

   A   3
   B   6
   C   4
   D   5


In [ ]:
import pandas as pd
df = pd.read_csv('closet.csv', names = ['Clothes'], sep = ',')
df.head()

,Clothes
0,"Camiseta,Jeans,Vestido,Sudadera"
1,"Chaqueta,Gorro,Vestido,Botas,Falda"
2,"Gabardina,Pantalón,Camiseta"
3,"Falda,Jeans,Chaqueta,Gorro"
4,"Vestido,Botas,Sudadera"


In [ ]:
data = list(df["Clothes"].apply(lambda x:x.split(",") ))
data

[['Camiseta', 'Jeans', 'Vestido', 'Sudadera'],
 ['Chaqueta', 'Gorro', 'Vestido', 'Botas', 'Falda'],
 ['Gabardina', 'Pantalón', 'Camiseta'],
 ['Falda', 'Jeans', 'Chaqueta', 'Gorro'],
 ['Vestido', 'Botas', 'Sudadera'],
 ['Pantalón', 'Camiseta', 'Chaqueta', 'Gorro'],
 ['Falda', 'Sudadera', 'Jeans', 'Chaqueta'],
 ['Gabardina', 'Botas', 'Vestido', 'Chaqueta', 'Camiseta'],
 ['Sudadera', 'Chaqueta', 'Pantalón', 'Falda', 'Gorro'],
 ['Jeans', 'Vestido', 'Botas', 'Chaqueta', 'Camiseta'],
 ['Vestido', 'Camiseta', 'Chaqueta'],
 ['Pantalón', 'Sudadera', 'Gorro'],
 ['Chaqueta', 'Falda', 'Jeans', 'Botas'],
 ['Camiseta', 'Vestido', 'Gorro', 'Sudadera', 'Pantalón'],
 ['Botas', 'Falda', 'Chaqueta', 'Jeans']]

In [ ]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
a = TransactionEncoder()
a_data = a.fit(data).transform(data)
df = pd.DataFrame(a_data,columns=a.columns_)
df = df.replace(False,0)
df

,Botas,Camiseta,Chaqueta,Falda,Gabardina,Gorro,Jeans,Pantalón,Sudadera,Vestido
0,0,True,0,0,0,0,True,0,True,True
1,True,0,True,True,0,True,0,0,0,True
2,0,True,0,0,True,0,0,True,0,0
3,0,0,True,True,0,True,True,0,0,0
4,True,0,0,0,0,0,0,0,True,True
5,0,True,True,0,0,True,0,True,0,0
6,0,0,True,True,0,0,True,0,True,0
7,True,True,True,0,True,0,0,0,0,True
8,0,0,True,True,0,True,0,True,True,0
9,True,True,True,0,0,0,True,0,0,True


In [ ]:
df = apriori(df, min_support = 0.2, use_colnames = True, verbose = 1)
df

Processing 64 combinations | Sampling itemset size 43


c:\Users\Daphne\AppData\Local\Programs\Python\Python311\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.4,(Botas)
1,0.466667,(Camiseta)
2,0.666667,(Chaqueta)
3,0.4,(Falda)
4,0.4,(Gorro)
5,0.4,(Jeans)
6,0.333333,(Pantalón)
7,0.4,(Sudadera)
8,0.466667,(Vestido)
9,0.333333,"(Botas, Chaqueta)"


In [ ]:
df_ar = association_rules(df, metric = "confidence", min_threshold = 0.8)
df_ar

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Botas),(Chaqueta),0.400000,0.666667,0.333333,0.833333,1.25,0.066667,2.0,0.333333
1,(Falda),(Chaqueta),0.400000,0.666667,0.400000,1.000000,1.50,0.133333,inf,0.555556
2,(Jeans),(Chaqueta),0.400000,0.666667,0.333333,0.833333,1.25,0.066667,2.0,0.333333
3,(Pantalón),(Gorro),0.333333,0.400000,0.266667,0.800000,2.00,0.133333,3.0,0.750000
4,"(Botas, Falda)",(Chaqueta),0.200000,0.666667,0.200000,1.000000,1.50,0.066667,inf,0.416667
5,"(Botas, Jeans)",(Chaqueta),0.200000,0.666667,0.200000,1.000000,1.50,0.066667,inf,0.416667
6,"(Gorro, Falda)",(Chaqueta),0.200000,0.666667,0.200000,1.000000,1.50,0.066667,inf,0.416667
7,"(Chaqueta, Jeans)",(Falda),0.333333,0.400000,0.266667,0.800000,2.00,0.133333,3.0,0.750000
8,"(Falda, Jeans)",(Chaqueta),0.266667,0.666667,0.266667,1.000000,1.50,0.088889,inf,0.454545
9,"(Sudadera, Gorro)",(Pantalón),0.200000,0.333333,0.200000,1.000000,3.00,0.133333,inf,0.833333


In [186]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

# Conjuntos de transacciones dados
transactions = [
    ['I1', 'I2', 'I3'],
    ['I2', 'I3', 'I4'],
    ['I4', 'I5'],
    ['I1', 'I2', 'I4'],
    ['I1', 'I2', 'I3', 'I5'],
    ['I1', 'I2', 'I3', 'I4']
]

# Instanciar TransactionEncoder
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df = pd.DataFrame(te_ary, columns=te.columns_)

# Aplicar el algoritmo Apriori
frequent_itemsets = apriori(df, min_support=0.5, use_colnames=True)

# Generar reglas de asociación
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)

# Mostrar las reglas de asociación generadas
display(rules)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(I2),(I1),0.833333,0.666667,0.666667,0.80,1.200,0.111111,1.666667,1.000000
1,(I1),(I2),0.666667,0.833333,0.666667,1.00,1.200,0.111111,inf,0.500000
2,(I1),(I3),0.666667,0.666667,0.500000,0.75,1.125,0.055556,1.333333,0.333333
3,(I3),(I1),0.666667,0.666667,0.500000,0.75,1.125,0.055556,1.333333,0.333333
4,(I2),(I3),0.833333,0.666667,0.666667,0.80,1.200,0.111111,1.666667,1.000000
5,(I3),(I2),0.666667,0.833333,0.666667,1.00,1.200,0.111111,inf,0.500000
6,(I4),(I2),0.666667,0.833333,0.500000,0.75,0.900,-0.055556,0.666667,-0.250000
7,"(I2, I1)",(I3),0.666667,0.666667,0.500000,0.75,1.125,0.055556,1.333333,0.333333
8,"(I2, I3)",(I1),0.666667,0.666667,0.500000,0.75,1.125,0.055556,1.333333,0.333333
9,"(I1, I3)",(I2),0.500000,0.833333,0.500000,1.00,1.200,0.083333,inf,0.333333
